In [20]:
import random
from IPython.display import display, clear_output
from ipywidgets import widgets, Button, Layout
import sympy as sym
import math
import time
import matplotlib.pyplot as plt
import pandas as pd
from prettytable import PrettyTable

p = None
A = None
B = None
E = 0
Z = 1
c = 2
d = 3
#JacobCoord(x, y, z, p, ToYac): Эта функция конвертирует точки между аффинными и Якобиановыми координатами.
#Если ToYac равно 1, 
#функция конвертирует из аффинных в Якобиановы координаты, иначе - наоборот.
def JacobCoord(x, y, z, p, ToYac):
    if ToYac == 1:
        if x == 0 and y == 1:
            return 1, 1, 0
        return mod(x/z^c, p), mod(y/z^d, p), mod(1/z, p)
    else:
        if x == 1 and y == 1 and z == 0:
            return 0, 1, 0
        return mod(x/z^c, p), mod(y/z^d, p) ,mod(1, p)
    
#double(X, Y, Z, A, p): Эта функция удваивает точку в якобиановых координатах. Функция возвращает координаты удвоенной точки.
def double(X, Y, Z, A, p):
    T1 = mod(X^2, p)
    T2 = mod(Y^2, p)
    T3 = mod(T2^2, p)
    C = mod(Z^2, p)
    
    S = mod(2*((X + T2)^2 - T1 - T3), p)
    
    M = mod(3*T1 + A*C^2, p)
    F = mod(M^2 - 2*S, p)
    
    X2 = F
    Y2 = mod(M*(S - F) - 8*T3, p)
    Z2 = mod((Y + Z)^2 - T2 - C, p)

    return X2, Y2, Z2

#addJacob(X1, Y1, Z1, X2, Y2, Z2, p): Эта функция добавляет 
#две точки в смешанных Якобианово-афинных координатах. Возвращает координаты результата.
def addJacob(X1, Y1, Z1, X2, Y2, Z2, p):
    if X2 == 0 and Y2 == 1:
        return X1, Y1, Z1
    
    T1 = mod(Z1^2, p)
    T2 = mod(Z2^2, p)
    U1 = mod(X1*T2, p)
    U2 = mod(X2*T1, p)
    
    S1 = mod(Y1*Z2*T2, p)
    S2 = mod(Y2*Z1*T1, p)
    H = mod(U2 - U1, p)
    I = mod((2*H)^2, p)
    
    J = mod(H*I, p)
    r = mod(2*(S2 - S1), p)
    V = mod(U1*I, p)
    
    X3 = mod(r^2 - J - 2*V, p)
    Y3 = mod(r*(V - X3) - 2*S1*J, p)
    Z3 = mod(((Z1 + Z2)^2 - T1 - T2)*H, p)

    return (X3, Y3, Z3)
#addPoints(P, Q, E): Эта функция принимает две точки P и Q в аффинных координатах, преобразует их в якобиановы координаты,
#добавляет их вместе с использованием функции addJacob, а затем преобразует результат обратно в аффинные координаты.
def addPoints(P, Q, E):
    z = randint(1, p-1)

    if P[0] == 0 and P[1] == 1:
        Xp, Yp, Zp = 1, 1, 0
    else:
        Xp, Yp, Zp = JacobCoord(P[0], P[1], z, p, 1)

    Xr, Yr, Zr = addJacob(Xp, Yp, Zp, Q[0], Q[1], 1, p)
    Xa, Ya, Za = JacobCoord(Xr, Yr, Zr, p, 0)

    return E(Xa, Ya, Za)

#doublePoint(P, E): Эта функция принимает точку P в аффинных координатах, преобразует ее в якобиановы координаты,
#удваивает ее с использованием функции double, а затем преобразует результат обратно в аффинные координаты.
def doublePoint(P, E):
    z = randint(1, p-1)
    
    Xp, Yp, Zp = JacobCoord(P[0], P[1], z, p, 1)
    Xd, Yd, Zd = double(Xp, Yp, Zp, A, p)
    Xa, Ya, Za = JacobCoord(Xd, Yd, Zd, p, 0)
    
    return E(Xa, Ya, Za)

#binaryMult(P, E, k): Эта функция выполняет двоичное
#умножение точки P на число k с использованием сложения и удвоения точек эллиптической кривой.
def binaryMult(P, E, k):
    Q = E(0, 1, 0)
    k_bin = k.binary()
    t = len(k_bin)
    i = t - 1
    while i >= 0 :
        if k_bin[i] == '1':
            Q = addPoints(P, Q, E)
        P = doublePoint(P, E)
        i -= 1
    return Q

def generate_y_on_elliptic_curve(E, x):
    y_sqr = x**3 + E.a4() * x + E.a6()  # Уравнение кривой для определения y^2
    if y_sqr.is_square():
        y = y_sqr.sqrt()
        return y
    else:
        return -5

def generate_random_point_on_elliptic_curve(E):
    while True:
        x = Integer(randrange(1, p))  # Случайное x в диапазоне [1, p-1]
        y_sqr = x**3 + E.a4() * x + E.a6()  # Уравнение кривой для определения y^2
        if y_sqr.is_square():
            y = y_sqr.sqrt()
            return E(x, y)
def generate_random_point_on_elliptic_curve2(E):
    while True:
        x = Integer(randrange(1, p_input.value))  # Случайное x в диапазоне [1, p-1]
        y_sqr = x**3 + E.a4() * x + E.a6()  # Уравнение кривой для определения y^2
        if y_sqr.is_square():
            y = y_sqr.sqrt()
            return E(x, y)

# def double_point(P, a, p):
#     if P is None:
#         return None

#     l = ((3 * P[0]**2 + a) * pow(2 * P[1], -1, p)) % p
#     x = (l**2 - 2 * P[0]) % p
#     y = ((l * (P[0] - x) - P[1]) % p)

#     return (x,y)


def addPoinnts(P, Q,p):

    if P is None:
        return Q
    if Q is None:
        return P


    l = ((Q[1] - P[1]) * pow(Q[0] - P[0], -1, p)) % p
    x = (l ** 2 - P[0] - Q[0]) % p
    y = ((l * (P[0] - x) - P[1]) % p)

    return (P+Q)


# def binary_multiplication(P, k, a, p):
#     if k == 0:
#         return None

#     k_binary = bin(k)[2:]
#     Q = None

#     for i in range(len(k_binary) - 1, -1, -1):
#         Q = double_point(Q, a, p)
#         if k_binary[len(k_binary) - 1-i] == '1':
#             Q = add_points(Q, P, a, p)

#     return Q


def jsf(k1, k2):
    k = {}
    k[1], k[2] = k1, k2
    l = 0
    d = {}
    d[1], d[2] = 0, 0
    K = {1:[], 2:[]}
    l_ = {}
    while (k[1] + d[1] > 0) or (k[2] + d[2] > 0):
        l_[1] = d[1] + k[1]
        l_[2] = d[2] + k[2]
        
        for i in range(1, 3):
            if mod(l_[i], 2) == 0:
                u = 0
            else:
                u = mod(l_[i], 4)
                if (mod(l_[i], 8) == 3 or mod(l_[i], 8) == -3) and mod(l_[3-i], 4) == 2:
                    u = -u
                
            if u == 3:
                K[i].append(-1)
            else:
                K[i].append(u)
        
        for i in range(1, 3):
            if 2*d[i] == 1 + K[i][l]:
                d[i] = 1 - d[i]
            k[i] = int(k[i]/2)
            
        l += 1
    
    return K


def precalculated(G, Q):
    list_points = {}
    
    list_points[5] = addPoints(G,Q,E)
    list_points[-5] = addPoints((-G),(-Q),E) #-G - Q
    list_points[2] = G
    list_points[3] = Q
    list_points[1] = addPoints((-G),Q,E) #-G + Q
    list_points[-1] = addPoints(G,(-Q),E) #G - Q
    list_points[-2] = -G
    list_points[-3] = -Q
    list_points[0] = 0
    
    return list_points


def JSF(k1, k2, G, Q):
    start_time = time.time()
    list_points = precalculated(G, Q)
    time_precalcul = time.time() - start_time
    
    start_time = time.time()
    K = jsf(k1, k2)
    d = len(K[1])
    R = E(0, 1, 0)
    
    for i in range(d - 1, -1, -1):
        #R = 2*R
        R = doublePoint(R, E)
        #R = R + int(K[1][i])*G + int(K[2][i])*Q
        M = list_points[Integer(2*K[1][i]) + Integer(3*K[2][i])]
        #print("Integer(2*K[1][i]) + Integer(3*K[2][i]) = ", Integer(2*K[1][i]) + Integer(3*K[2][i]))
        #print("M = ", M)
        if (Integer(2*K[1][i]) + Integer(3*K[2][i]))!=0:
            R = addPoinnts(R, M,p)
        #R = R + list_points[int(2*K[1][i]) + int(3*K[2][i])]
    
    return R, time.time() - start_time, time_precalcul



def run_big_test(b):
    global p, A, B, E
    n = 1024
    k_bits = [32, 64, 128, 256, 512,1023]
    #num_passes = 5
    
   
    JSF_times = []
    Stupid_Solution_Times = []
    
    p = random_prime(2^n - 1, lbound=2^(n-1))
    A, B = 0, 0
    E = None
    while not E:
        A = randint(2, p - 2)
        B = randint(2, p - 2)
        try:
            E = EllipticCurve(GF(p), [A, B])
        except ValueError:
            E = None
    P1 = generate_random_point_on_elliptic_curve(E)
    P2 = generate_random_point_on_elliptic_curve(E)
    P3 = generate_random_point_on_elliptic_curve(E)
    print("p (bit) =", len(p.binary()))
    print("E = ", E)
    print("P1 = ", P1)
    print("P2 = ", P2)
    print("P3 = ", P3)
    for k in k_bits:
        #print('k=',k)
        Stupid_Solution_Times_P1_P2 = 0
        Stupid_Solution_Times_P1_P3 = 0
        Stupid_Solution_Times_P2_P3 = 0
        JSF_P1_P2_times = 0
        JSF_P1_P3_times = 0
        JSF_P2_P3_times = 0
        k1 = random_prime(2^k - 1, lbound=2^(k-1))
        #print("k1 (bit) =", len(k1.binary()))
        k2 = random_prime(2^k - 1, lbound=2^(k-1))
        #print("k2 (bit) =", len(k2.binary()))
        #print()
        
        start_time = time.time()
        P11 = binaryMult(P1, E, k1) 
        P22 = binaryMult(P2, E, k2) 
        PP = addPoints(P11,P22,E)
        Stupid_Solution_Times_P1_P2 = time.time() - start_time  
        #print("Stupid_Solution_Times_P1_P2 = ", PP)
        
        start_time = time.time()
        P11 = binaryMult(P1, E, k1) 
        P33 = binaryMult(P3, E, k2) 
        PP = addPoints(P11,P33,E)
        Stupid_Solution_Times_P1_P3 = time.time() - start_time
        
        start_time = time.time()
        P22 = binaryMult(P2, E, k1) 
        P33 = binaryMult(P3, E, k2) 
        PP = addPoints(P22,P33,E)
        Stupid_Solution_Times_P2_P3 = time.time() - start_time
        
        Stupid_Solution_Times.append((Stupid_Solution_Times_P1_P2 + Stupid_Solution_Times_P1_P3 + Stupid_Solution_Times_P2_P3 / 3))
        
        PP, times, times2 = JSF(k1, k2, P1, P2)
        JSF_P1_P2_times = times
        #print("JSF_P1_P2 = ", PP)
        #print()
        
        PP, times, times3 = JSF(k1, k2, P1, P3)
        JSF_P1_P3_times = times
        
        PP, times, times4 = JSF(k1, k2, P2, P3)
        JSF_P2_P3_times = times
        if k == 32:
            All_time = times2+times3+times4
            print("Время предвычислений: ", All_time)
        
        JSF_times.append((JSF_P1_P2_times + JSF_P1_P3_times + JSF_P2_P3_times) / 3 )

    # Create a DataFrame with results
    results = pd.DataFrame({
        'k_bits': k_bits,
        'Stupid_Solution Time': Stupid_Solution_Times,
        'JSF Time': JSF_times
    })
    
    # Display the table
    print(results)
    
    # Создайте график
    plt.figure(figsize=(30, 15))

    # Отображаем данные на графике
    plt.plot(k_bits, Stupid_Solution_Times, marker='o', label='Stupid_Solution')
    plt.plot(k_bits, JSF_times, marker='o', label='JSF')
    

    # Добавляем заголовок и метки осей
    plt.title('Время выполнения программы', fontsize=24)
    plt.xlabel('Разрядность k', fontsize=18)
    plt.ylabel('Время (секунды)', fontsize=18)

    # Добавляем легенду с максимальным увеличением размера шрифта
    plt.legend(fontsize=18)

    # Настройка меток оси X
    plt.xticks(k_bits, fontsize=20)
    # Настройка меток оси Y
    plt.yticks(fontsize=20)

    # Отобразить график
    plt.grid()
    plt.show()  

def test():
    global p,A,B,E,Xg,Yg,Xq,Yq,k1,k2
    p = 115792089237316193816632940749697632408932724669461758699102671622853342616127
    A = 10001090682862736796505780343107022911835438229291751861888571869223050424594
    B = 45264756867680555803214833811970559410867867042681754140959938453766481155105
    Xg = 34131522064967883044818697086599304935071739419701303653954060188761196377435
    Yg = 22161743160862324650134218095423264926581638655388631138889754360346138331114
    Xq = 36165359026576437742151345460371859975651866220385050258939633540903065959717
    Yq = 36943478257105992475915633328919578144889502533328778740461803868682833354169
    k1 = 440044004412345678900987654321
    k2 = 44^16

    E = EllipticCurve(GF(p), [A, B])
    G = E([Xg, Yg])
    Q = E([Xq, Yq])
    lens = [32, 64, 128, 256, 512, 1024, 2048, 4096]

    for i in lens:
        k1 = random_prime(2^i - 1, lbound=2^(i-1))
        k2 = random_prime(2^i - 1, lbound=2^(i-1))
        print("k1 (bit) =", len(k1.binary()))
        print("k2 (bit) =", len(k2.binary()))
        #k1 = random.randint(2^i, 2^(i + 1))
        #k2 = random.randint(2^i, 2^(i + 1))
        P, times = JSF(k1, k2, G, Q)
        print("JSF = ", P)
        
        start_time = time.time()
        P1 = binaryMult(G, E, k1) 
        P2 = binaryMult(Q, E, k2) 
        P = addPoints(P1, P2,E)
        print("Sage = ", P)
        print("Sage =", time.time() - start_time)
        print("JSF time =", times)
        print()
      
    



def run_lab_variant(b):
    global p,A,B,E,Xg,Yg,Xq,Yq,k1,k2
    p = 115792089237316193816632940749697632408932724669461758699102671622853342616127
    A = 10001090682862736796505780343107022911835438229291751861888571869223050424594
    B = 45264756867680555803214833811970559410867867042681754140959938453766481155105
    Xg = 34131522064967883044818697086599304935071739419701303653954060188761196377435
    Yg = 22161743160862324650134218095423264926581638655388631138889754360346138331114
    Xq = 36165359026576437742151345460371859975651866220385050258939633540903065959717
    Yq = 36943478257105992475915633328919578144889502533328778740461803868682833354169
    k1 = 440044004412345678900987654321
    k2 = 44^16

    E = EllipticCurve(GF(p), [A, B])
    G = E([Xg, Yg])
    Q = E([Xq, Yq])

    print(E)
    print()
    print("JSF =", JSF(k1, k2, G, Q)[0])
    print("Sage =", k1*G + k2*Q)
        
def run_test(b):
    print('test')
    test()
    print("End")
    
E1 = None
def generate_random_p(b):
    #print("p (bit) = ",p_bit_input )
    p = random_prime(2^p_bit_input.value - 1, lbound=2^(p_bit_input.value-1))
    p_input.value = p
    
def generate_random_AB(b):
    global E1
    A1, B1 = 0, 0
    E1 = None
    while not E1:
        A1 = randint(2, p_input.value - 2)
        B1 = randint(2, p_input.value - 2)
        try:
            E1 = EllipticCurve(GF(p_input.value), [A1, B1])
        except ValueError:
            E1 = None
    A_input.value = A1
    B_input.value = B1
    

def generate_random_X1(b):
    #print("1", E1)
    random_X1 = generate_random_point_on_elliptic_curve2(E1)
    #print("random_X1 = ", random_X1)
    Xg_input.value = random_X1[0]
    
def generate_random_X2(b):
    #print("1", E1)
    random_X2 = generate_random_point_on_elliptic_curve2(E1)
    #print("random_X1 = ", random_X1)
    Xg2_input.value = random_X2[0]
    
def generate_random_k1(b):
    #print("p (bit) = ",p_bit_input )
    k1 = random_prime(2^k_bit_input.value - 1, lbound=2^(k_bit_input.value-1))
    k1_input.value = k1
    
def generate_random_k2(b):
    #print("p (bit) = ",p_bit_input )
    k2 = random_prime(2^k_bit_input.value - 1, lbound=2^(k_bit_input.value-1))
    k2_input.value = k2
    
def generate_random_start(b):
    global p, A, B, E
    p = Integer(p_input.value)
    A = A_input.value
    B = B_input.value
    
    k1 = Integer(k1_input.value)
    k2 = Integer(k2_input.value)
    
    E = EllipticCurve(GF(p), [A, B])
    print()
    print("E = ", E)
    print("p =", p)
    print("p (bit) =", len(p.binary()))

    Xg = Xg_input.value
    Yg = generate_y_on_elliptic_curve(E, Xg)
    P1 = E(Xg, Yg)
    
    Xg2 = Xg2_input.value
    Yg2 = generate_y_on_elliptic_curve(E, Xg2)
    P2 = E(Xg2, Yg2)
    print("P1 = ", P1)
    print("P2 = ", P2)
    print("P1* ", k1,"+ P2*", k2)
    
    print("JSF =", JSF(k1, k2, P1, P2)[0])
    print("Sage =", k1*P1 + k2*P2)
    
    P, times, time_pre = JSF(k1, k2, P1, P2)
    start_time = time.time()
    P11 = binaryMult(P1, E, k1) 
    P22 = binaryMult(P2, E, k2) 
    PP = addPoints(P11, P22,E)
    print("Sage time =", time.time() - start_time)
    print("JSF time =", times)
    print("Время предвычислений: ", time_pre)

    


p_bit_input = widgets.IntText(description='Разр. p: ',layout=widgets.Layout(width="300px", height="50px"))
p_input = widgets.IntText(description='Введите p: ')
generate_button = widgets.Button(description='Сгенерировать p', layout=widgets.Layout(width="300px", height="50px"))
generate_button.on_click(generate_random_p)
  

A_input = widgets.IntText(description='A = ')
B_input = widgets.IntText(description='B = ')

generate_button_AB = widgets.Button(description='Сгенерировать A и B', layout=widgets.Layout(width="300px", height="50px"))
generate_button_AB.on_click(generate_random_AB)

Xg_input = widgets.IntText(description='X1 = ')

generate_button_X = widgets.Button(description='Сгенерировать X1', layout=widgets.Layout(width="300px", height="50px"))
generate_button_X.on_click(generate_random_X1)

Xg2_input = widgets.IntText(description='X2 = ')
generate_button_X2 = widgets.Button(description='Сгенерировать X2', layout=widgets.Layout(width="300px", height="50px"))
generate_button_X2.on_click(generate_random_X2)



k_bit_input = widgets.IntText(description='Разр. k: ',layout=widgets.Layout(width="300px", height="50px"))
k1_input = widgets.IntText(description='Введите k1: ')
generate_button_k1 = widgets.Button(description='Сгенерировать k1', layout=widgets.Layout(width="300px", height="50px"))
generate_button_k1.on_click(generate_random_k1)

k2_input = widgets.IntText(description='Введите k2: ')
generate_button_k2 = widgets.Button(description='Сгенерировать k2', layout=widgets.Layout(width="300px", height="50px"))
generate_button_k2.on_click(generate_random_k2)

lets_go = widgets.Button(description='Запустить алгоритмы', layout=widgets.Layout(width="300px", height="50px"))
lets_go.on_click(generate_random_start)
    

button_lab = widgets.Button(description="1. Запустить вариант из лабораторной", layout=widgets.Layout(width="500px", height="50px"))
button_test = widgets.Button(description="2. Запустить тест (t и p близки (32,64,128...))", layout=widgets.Layout(width="500px", height="50px"))
button_test_big = widgets.Button(description="3. Запустить тест (среднее время для 3 точек)", layout=widgets.Layout(width="500px", height="50px"))

button_lab.on_click(run_lab_variant)
button_test.on_click(run_test)
button_test_big.on_click(run_big_test)

#display(button_lab, button_test,button_test_big,p_bit_input,p_input,generate_button_p, A_input,B_input,generate_button_AB,Xg_input,generate_button_X,k_bit_input,k_input,generate_button_k,lets_go)
display(button_lab, button_test,button_test_big,p_bit_input,p_input,generate_button,A_input,B_input,generate_button_AB,Xg_input,generate_button_X,Xg2_input,generate_button_X2,k_bit_input,k1_input,generate_button_k1,k2_input,generate_button_k2,lets_go)
#test(G, Q)


Button(description='1. Запустить вариант из лабораторной', layout=Layout(height='50px', width='500px'), style=…

Button(description='2. Запустить тест (t и p близки (32,64,128...))', layout=Layout(height='50px', width='500p…

Button(description='3. Запустить тест (среднее время для 3 точек)', layout=Layout(height='50px', width='500px'…

IntText(value=0, description='Разр. p: ', layout=Layout(height='50px', width='300px'))

IntText(value=0, description='Введите p: ')

...

Button(description='Сгенерировать k1', layout=Layout(height='50px', width='300px'), style=ButtonStyle())

IntText(value=0, description='Введите k2: ')

Button(description='Сгенерировать k2', layout=Layout(height='50px', width='300px'), style=ButtonStyle())

Button(description='Запустить алгоритмы', layout=Layout(height='50px', width='300px'), style=ButtonStyle())


E =  Elliptic Curve defined by y^2 = x^3 + 1779772268*x + 1604751060 over Finite Field of size 4035595313
p = 4035595313
p (bit) = 32
P1 =  (1775922215 : 1574846898 : 1)
P2 =  (3428391586 : 290255483 : 1)
P1*  1073337829 + P2* 935151479
JSF = (72424253 : 3561484194 : 1)
Sage = (72424253 : 3561484194 : 1)
Sage time = 0.030763626098632812
JSF time = 0.01217031478881836
Время предвычислений:  0.0015256404876708984
